In [2]:
%%capture
## compile PyRoss for this notebook
import os
owd = os.getcwd()
os.chdir('../')
%run setup.py install
os.chdir(owd)

ModuleNotFoundError: No module named 'Cython'

In [2]:
import numpy as np
import pyross
import matplotlib.pyplot as plt

In [ ]:
# age structure of the population

N =  1000            # total population

M  = 2               # age groups
Ni = np.zeros((M))   # population in each group
fi = np.zeros((M))   # fraction of population in age age group

fi = np.array((0.25, 0.75)) 

for i in range(M):
    Ni[i] = fi[i]*N

# contact structure of the population
C = np.array(([18., 9.], [3., 12.]))

# matrix for linearised dynamics
L = np.zeros((M, M))
L[0,0] = C[0,0]*Ni[0]/Ni[0]
L[0,1] = C[0,1]*Ni[0]/Ni[1]
L[1,0] = C[1,0]*Ni[1]/Ni[0]
L[1,1] = C[1,1]*Ni[1]/Ni[1]

# epidemic parameters
alpha = 1.              # symptomatic fraction
gamma = 1./3            # recovery rate 

# largest eigenvalue of L is r_0
r_0 = np.max(np.linalg.eigvals(L))

# obtain beta from r_0
beta = r0*gamma
  
# initial conditions    
Ia_0 = np.array((1,1))
Is_0 = np.array((1,1))
R_0  = np.array((0,0))
S_0 = Ni - (Ia_a + Is_0 + R_0)

In [ ]:
fsa=0
model = pyross.models.SIR(S_0, Ia_0, Is_0, alpha, beta, gamma, fsa, M, Ni)

In [ ]:
Tf=200; filename='this.mat'
model.simulate(Tf, C, filename)

In [ ]:
from scipy.io import loadmat
data=loadmat(filename)
IaK = data['X'][:,M].flatten()
IaA = data['X'][:,M+1].flatten()
t = data['t'][0] 

In [ ]:
fig = plt.figure(num=None, figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')
plt.rcParams.update({'font.size': 22})


plt.fill_between(t, 0, IaK/Ni[0], color="#348ABD", alpha=0.3)
plt.plot(t, IaK/Ni[0], '-', color="#348ABD", label='$Kids$', lw=4)

plt.fill_between(t, 0, IaA/Ni[1], color='#A60628', alpha=0.3)
plt.plot(t, IaA/Ni[1], '-', color='#A60628', label='$Adults$', lw=4)

plt.legend(fontsize=26); plt.grid() 
plt.autoscale(enable=True, axis='x', tight=True)

In [ ]:
plt.loglog(t, IaK/Ni[0], '-', color="#348ABD", label='$Kids$', lw=4)
plt.loglog(t, IaA/Ni[1], '-', color="#348ABD", label='$Kids$', lw=4)

In [ ]:
from scipy.linalg import expm


Nt=5#int(np.size(t)/4)
pp = np.zeros((Nt, M))
for i in range(Nt):
    pp[i,:]=expm(gamma*(L-1)*i)[0,0]
    

In [ ]:
pp